# CNN Case Study

## Introduction

**Learning Objectives**
- How to classify medical images
- Using the confusion matrix to evaluate model performance
- Using the ROC curve to evaluate model performance per class
- Using F1 score to evaluate model performance per class

To further demonstrate **CNN's** capabilities, we will apply it to a medical image classification problem. Specifically, we will use the PathMNIST dataset, which contains histological image samples.

## The PathMNIST dataset

The PathMNIST dataset contains 107,180 colon pathology samples with 9 classes. The dataset is split into a training set of 96,492 samples and a validation set of 10,688 samples. The PathMNIST is based on a prior study for predicting survival from colorectal cancer histology slides, providing a dataset of 100,000 non-overlapping image patches from haematoxylin & eosin-stained histological images, and a test dataset of 7,180 image patches from a different clinical center. Having the test dataset sampled from another clinic is a *great* example of a rigorous test set, as the model will not be picking up any clinic-specific artifacts.

The dataset is comprised of 9 types of tissues, resulting in a multi-class classification task.

The classes are:

In [ ]:
real_labels = {
            0: "adipose",
            1: "background",
            2: "debris",
            3: "lymphocytes",
            4: "mucus",
            5: "smooth muscle",
            6: "normal colon mucosa",
            7: "cancer-associated stroma",
            8: "crc adenocarcinoma epithelium"
        }

NOTE BOX

The dataset is available from the [MedMNIST](https://medmnist.com/) website, where there are several more image-based, medical datasets available for the  purpose of [testing new models](https://www.nature.com/articles/s41597-022-01721-8). See the image below for the overview of the catalogue. 

The PathMNIST dataset was created and made available by *Jakob Nikolas Kather, Johannes Krisam, et al., "Predicting survival from colorectal cancer histology slides using deep learning: A retrospective multicenter study," PLOS Medicine, vol. 16, no. 1, pp. 1–22, 01 2019.*

The  `data_flag` can be used to choose any of the following:

![MED-MNIST catalogue](./images/medmnistv2.jpg)

## Setup

In [ ]:
import torch # the PyTorch module
import torch.nn as nn # the neural network module
import torch.nn.functional as F # the functional module
import torch.optim as optim # the optimiser module

import warnings
warnings.filterwarnings("ignore") # ignore warnings

# Import the MNIST dataset
from torchvision import transforms

import matplotlib.pyplot as plt
import numpy as np

# fix the numpy and torch seeds for reproducibility:
SEED = 999
np.random.seed(SEED)
torch.manual_seed(SEED);

In [ ]:
# import medmnist, the medical image dataset
import medmnist
from medmnist import INFO

Using the `INFO` dictionary, we can access the metadata of the dataset to confirm the details of the dataset.

Unlike the MNIST images, these pathology samples are coloured and have **3 channels** (red, green, blue) instead of **1 channel** (grayscale). Each channel will be processed by a convolutional layer, and the outputs will be concatenated to form a single feature map. The feature map will then be passed through a fully connected layer to classify the image. The output of the fully connected layer will be a vector of length 9, where each element represents the probability that the image belongs to a particular class.

In [ ]:
data_flag = 'pathmnist'

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

print(f"The learning task is {task}")
print(f"The number of channels is {n_channels}")
print(f"The number of classes is {n_classes}")

As previously, we will download the dataset, transform it and create a dataloader:

In [ ]:
download = True
BATCH_SIZE = 128

DataClass = getattr(medmnist, info['python_class'])

# preprocessing
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5]) # normalise the data with standard values
])

train_loader = torch.utils.data.DataLoader(
    DataClass(split='train', 
                transform=data_transform, 
                download=download, 
                root='data'),
    batch_size=BATCH_SIZE, shuffle=True)

# load the data
test_loader = torch.utils.data.DataLoader(
    DataClass(split='test', 
                transform=data_transform, 
                download=download, 
                root='data'),
    batch_size=2*BATCH_SIZE, shuffle=True)

In [ ]:
# Some of the images in the dataset:
train_loader.dataset.montage(length=25)

## Defining the CNN

### The CNN Architecture

For this pathology image classification task, we've designed a deeper CNN architecture that builds upon the principles we learned with MNIST, but adapted for the increased complexity of medical images.

Our network, called `CNN_Pathology`, is structured as a series of five convolutional blocks followed by a fully connected classifier. Unlike the simple MNIST images, pathology samples contain rich colour information across three channels (red, green, and blue), requiring our network to process more complex visual patterns. Compared to the CNN in the [previous page](./pytorch-implementation.html), we have defined unique sequential runs per layer, rather than having repeated maxpooling every layer.

**First Layers:**

The network begins with two convolutional layers that work together to extract low-level features. The first layer uses 16 filters with 3×3 kernels to detect basic patterns like edges and colour gradients in the input image. Each convolutional operation is immediately followed by **batch normalisation**, a technique that stabilises training by normalising the activations, and a **ReLU activation function** to introduce non-linearity.

The second layer maintains 16 filters but adds a **max pooling layer** with a 2×2 kernel. This pooling operation reduces the image's spatial dimensions by half

**The Middle Layers:**

Layers three and four increase the number of filters to 64, allowing the network to learn more complex, abstract features. These layers work without pooling, maintaining the spatial resolution while building richer feature representations.

**The Fifth Layer:**

The fifth convolutional block again uses 64 filters but introduces **padding** (adding a border of pixels around the input). This padding ensures that the spatial dimensions are preserved through the convolution, preventing information loss at the edges. After this final feature extraction, another max pooling layer reduces the spatial dimensions once more, producing compact feature maps of size 4×4.

**The Fully Connected Classifier:**

After the convolutional layers have extracted hierarchical features, the network flattens the 64 feature maps (each 4×4) into a single vector of 1,024 values (We will go through this calculation in detail). This vector is then passed through three fully connected layers that progressively reduce the dimensionality: 1,024 → 128 → 128 → 9 (the number of tissue classes).

Critically, we include **dropout layers** with a 60% dropout rate between the fully connected layers. Dropout randomly deactivates neurons during training, forcing the network to learn more generalisable features rather than solely the training data. This is one of the primary defences against **overfitting**. This solution works for both standard neural networks, as well as CNNs.

The final layer outputs 9 values, one for each tissue type in our classification task. These raw scores (logits) will be converted to probabilities by the loss function during training.

In [ ]:
# A CNN_Pathology model class but with dropout layers to avoid overfitting:

class CNN_Pathology(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()

        self.layer1 = nn.Sequential(
            # Input image: 28x28, Output image: 28 - 3 + 1 = 26x26
            nn.Conv2d(in_channels, 16, kernel_size=3), 
            nn.BatchNorm2d(16),
            nn.ReLU())

        self.layer2 = nn.Sequential(
            # Input image: 26x26, Output image: (26 - 3 + 1) / 2 = 12x12
            nn.Conv2d(16, 16, kernel_size=3), 
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer3 = nn.Sequential(
            # Input image: 12x12, Output image: 12 - 3 + 1 = 10x10
            nn.Conv2d(16, 64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU())
        
        self.layer4 = nn.Sequential(
            # Input image: 10x10, Output image: 10 - 3 + 1 = 8x8
            nn.Conv2d(64, 64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU())

        self.layer5 = nn.Sequential(
            # Input image: 8x8, Output image: (8 - 3 + 1 + 2(1)) / 2 = 4x4
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.fc = nn.Sequential(
            nn.Linear(64 * 4 * 4, 128), # 64 feature maps x spatial dimensions (4x4)
            nn.ReLU(),
            # a dropout layer is added here to avoid overfitting
            nn.Dropout(p=0.6),
            nn.Linear(128, 128),
            nn.ReLU(),
            # a dropout layer is added here to avoid overfitting
            nn.Dropout(p=0.6),
            nn.Linear(128, num_classes))

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = x.reshape(x.size(0), -1)
        x = self.fc(x)
        return x

### Calculating the spatial image dimensions

Keeping track of the feature map size is an important step in defining the architecture of a CNN and one that feels unintuitive at first. The above is a good example of this with changing applications of max pooling and padding. 

![Convolution Arithmetic](./images/convolution_arithmetic.png)

**Calculating the Feature Map Size:**

Starting size: PathMNIST images are 28×28 (standard for MNIST)

**First layer:**
```
nn.Conv2d(in_channels, 16, kernel_size=3)
```

The first layer uses a 3×3 kernel with **no padding**. Without padding, the kernel cannot be centered on the edge pixels of the image, as it would extend beyond the image boundaries. This means we lose one pixel on each side of the image. Using the formula **Output = Input - Kernel + 1**, we get:
$$ 28 - 3 + 1 = 26 $$
reducing the spatial dimensions of the feature map to **26×26**.

**Second layer:**

```
nn.Conv2d(16, 16, kernel_size=3)
```

The beginning of the second layer is a repeat of the first. Applying the same formula, we get:
$$ 26 - 3 + 1 = 24 $$
reducing the spatial dimensions of the feature map to **24×24**.

Including in the second layer is a `nn.MaxPool2d(2, stride=2)`, which further reduces the spatial dimensions by half to **12×12**.

**Third layer:**

```
nn.Conv2d(16, 64, kernel_size=3)
```

The third layer is the same as the first:
$$ 12 - 3 + 1 = 10 $$
reducing the spatial dimensions of the feature map to **10×10**.

**Fourth layer:**

```
nn.Conv2d(64, 64, kernel_size=3)
```

Again:
$$ 10 - 3 + 1 = 8 $$
reducing the spatial dimensions of the feature map to **8×8**.

**Fifth layer:**

```
nn.Conv2d(64, 64, kernel_size=3, padding=1) + MaxPool2d(2, stride=2)
```

The fifth layer adds padding. This means the kernel can be centered on the edge pixels of the image, as it extends beyond the image boundaries. Using the formula **Output = Input - Kernel + 1 + 2(Padding)**, we get:
$$ 8 - 3 + 1 + 2(1) = 8 $$
maintaining the spatial dimensions of the feature map at **8×8**.

Afterwards the max pooling is applied, reducing the spatial dimensions by half to **4×4**.


**Final calculation:**

Now we know the final feature map size is **4×4**, we multiply this by the number of kernels (channels) to get the total number of parameters in the final feature map. In this case, we have **64** kernels, so the final feature map has a total of $4\times4\times64 = 1024$ parameters.

## Training the model

### Setup

As before we can initialise the model and move it to the device.

In [ ]:
# Define the device and model
if torch.cuda.is_available():
    # if there are multiple GPUs, choose the first one
    device = torch.device("cuda") # Defines the device you want to have data moved to
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print(f"Using: {device} (Cuda)")

elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"Using: {device} (Apple Silicon)")

else:
    print("No GPU detected! Falling back to CPU")
    # If the GPU is not available, use the CPU
    device = torch.device("cpu")

model = CNN_Pathology(in_channels=n_channels, num_classes=n_classes)
model = model.to(device)

For the sake of time we will only train the model for **5 epochs**, although typically you would train for many more, say **> 100 epochs**. 

TIP BOX

Running this for 20 epochs in a CodeSpace took approximately 35 minutes.

In [ ]:
# Set the number of epochs, learning rate, and other parameters
n_epochs = 5
learning_rate = 0.001

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimiser, the weight decay is added to the optimiser to prevent overfitting
optimiser = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.01)

### Preventing overfitting

As our model has become far more complex, adding many more parameters, it becomes prone to **overfitting** to the training data. We have already introduced `Dropout`, which turns off some of the parameters to reduce the model's capacity. Now we will look at **over training** our model by running it for too long.

First we have the `ReduceLROnPlateau` scheduler, which reduces the learning rate when the validation loss plateaus (stops improving). This prevents overfitting by slowing down the learning process when the model is no longer making significant progress, allowing it to fine-tune its parameters more carefully rather than making large updates that might cause it to memorise training data. When the test loss stops decreasing, the scheduler automatically reduces the learning rate by a factor (in this case, 0.1), giving the model a chance to find better solutions with smaller, more precise weight updates.

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

patience = 5 # The number of epochs to wait before reducing the learning rate
lr_factor = 0.1 # The factor by which to reduce the learning rate

# Define the learning rate scheduler; this will reduce the learning rate when the validation loss plateaus
lr_scheduler = ReduceLROnPlateau(optimiser, mode='min', factor=lr_factor, patience=patience, verbose=True)

Additionally, we can implement **early stopping**, by *breaking* the training loop when the test loss stops improving. We do this by counting the number of epochs without improvement. This prevents the model from continuing to train when it's no longer learning useful patterns, saving computational resources and ensuring we keep the best-performing version of the model rather than an overtrained one.

In [ ]:
best_test_loss = float('inf')      # Track the lowest test loss seen so far (starts at infinity)
epoch_patience = 10                # The number of epochs without improvement to stop at
epochs_no_improve = 0              # Count consecutive epochs without improvement, break if == patience (5)

NOTE BOX

If you are running this notebook yourself, increase the number for `n_epochs` so that it runs for more than **5**.

### The training loop

We will not make use of the previously defined functions as we have implemented the training loop explicitly below.

In [ ]:
# 20 epochs took 32m:50s in a codespace

# Track the training and test loss
train_losses, test_losses = [], [] 

# Train the model
for epoch in range(n_epochs):

    # Track the training and test loss
    train_loss = 0.0
    test_loss = 0.0

    # Train the model
    model.train()
    for data, target in train_loader:

        # move tensors to GPU if CUDA is available
        data, target = data.to(device), target.to(device)
        # Clear the gradients of all optimized variables
        optimiser.zero_grad()
        # Forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        target = target.squeeze().long()
        # Calculate the batch loss
        loss = criterion(output, target)
        # Backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # Perform a single optimization step (parameter update)
        optimiser.step()
        # Update training loss
        train_loss += loss.item() * data.size(0)


    # Compute test accuracy
    correct = 0
    total = 0

    # Validate the model
    model.eval()
    # Turn off gradients for validation
    with torch.no_grad():
        for data, target in test_loader:
            # Move tensors to GPU if CUDA is available
            data, target = data.to(device), target.to(device)
            # Forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            target = target.squeeze().long() # .long() ensures the target is an integer 
            # Calculate the batch loss
            loss = criterion(output, target)
            # Update average test loss
            test_loss += loss.item() * data.size(0)
            # compare predictions to true label
            _, pred = torch.max(output, 1)
            correct += (pred == target).sum().item()
            total += target.size(0)

    # Calculate average losses and update
    train_loss = train_loss / len(train_loader.sampler)
    train_losses.append(train_loss)
    test_loss = test_loss / len(test_loader.sampler)
    test_losses.append(test_loss)

    # Update the learning rate scheduler,
    # so it is aware of the current validation loss
    lr_scheduler.step(test_loss)

    # Save the best model based on whether test loss is lower
    if test_loss < best_test_loss:

        best_test_loss = test_loss
        # Save the model parameters for this epoch
        best_model_params = model.state_dict().copy() # extract the model parameters
        epochs_no_improve = 0 # reset the counter

    else:
        # Update the number of epochs without improvement
        epochs_no_improve += 1
        # Check early stopping condition and stop training if loss did not improve in patience number of epochs
        if epochs_no_improve == epoch_patience:
            print(f'Early stopping after {epoch_patience} epochs without improvement.')
            break

    print(f'Epoch: {epoch+1} | Train Loss: {train_loss:.4f} | Test Loss: {test_loss:.4f} | Test Accuracy: {100 * correct / total:.2f}%')

## Saving the model

If we are happy with the model that performed the best on the test dataset we can save its parameters for use later.

In [ ]:
# Load the best model's parameters back into the model
model.load_state_dict(best_model_params)

# Save the best model
torch.save(model.state_dict(), './data/best_model.pth')

Due to the complexity of the data and the model we should train the model for much longer, as can be seen in the plot of the losses below:

In [ ]:
# plot the training and test loss
x_epochs = range(1, len(train_losses) + 1)
plt.plot(x_epochs, train_losses, label='Training loss')
plt.plot(x_epochs, test_losses, label='Validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.xticks(range(1, len(x_epochs) + 1)) 
plt.legend(frameon=False)
plt.title('Training and Validation Loss for the CNN on Pathology Images')
plt.show()

## Analysing the model

### Visual inspection

Even after only 5 epochs, the model has a 75% accuracy on the validation set. Looking at the predictions given the images we can see that the model is performing well on a difficult task.

In [ ]:
# set the model to evaluation mode before making predictions
model.eval()

# obtain one batch of test images
dataiter = iter(test_loader)
images, labels = next(dataiter)
images = images.to(device)
labels = labels.to(device)

# get sample outputs
with torch.no_grad():
    output = model(images)

# convert output probabilities to predicted class
_, preds = torch.max(output, 1) # predicted class == highest logit

# prep images for display
images = images.cpu().numpy()

# plot the images in the batch, along with predicted and true labels
fig = plt.figure(figsize=(18, 4))
        
for idx in np.arange(10):
    ax = fig.add_subplot(2, 5, idx+1, xticks=[], yticks=[])
    plt.imshow(np.clip(np.transpose(images[idx], (1, 2, 0)), 0, 1))
    ax.set_title(f"{real_labels[int(preds[idx].cpu().numpy())]}", color=("green" if preds[idx]==labels[idx] else "red"))

## Comparing classes

Accuracy and visual plotting does not tell us much about *where* the model is going wrong. It is common for a model to struggle with specific classes, rather than underperforming across the board. Three of the easiest analytical tools for comparing class performance include the **confusion matrix**, **ROC** curves and their accompanying **AUC** score, and the **F1** score, which is a balance between precision and recall.

First we will need the predictions from our best performing model (the one with the lowest loss) on the test set, and their corresponding true labels.

In [ ]:
# First, store all the test images as one tensor:
test_images = []
test_labels = []

for data, target in test_loader:
    test_images.append(data)
    test_labels.append(target)

# concatenate the test images and labels
test_images = torch.cat(test_images)
# .squeeze() converts shape [N, 1] → [N], which is required by sklearn metrics
test_labels = torch.cat(test_labels).squeeze()

# move tensors to GPU if CUDA is available
test_images = test_images.to(device)
test_labels = test_labels.to(device)

# get sample outputs — use no_grad() to avoid storing unnecessary gradients
with torch.no_grad():
    output = model(test_images)

# convert output probabilities to predicted class
_, all_preds = torch.max(output, 1)

### Confusion matrix:

The **confusion matrix** evaluates the number of correct and incorrect predictions made by the model compared to the actual observations in the data. Typically this is visualised with a colour bar to quickly highlight which classes are being predicted correctly or wrongly.

A binary classification **confusion matrix** will have the following components in **four** squares:
- True Positive (TP): Correctly predicted positive cases. (top left)
- True Negative (TN): Correctly predicted negative cases. (bottom right)
- False Positive (FP): Incorrectly predicted as positive, but are actually negative (Type I error). (bottom left)
- False Negative (FN): Incorrectly predicted as negative, but are actually positive (Type II error). (top right)

`Sklearn` gives us two handy functions for calculating and plotting the confusion matrix:

In [ ]:
# Define a function to plot the confusion matrix

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def plot_confusion_matrix(y_true, y_pred, classes=None, normalise=False, colmap=True, ax=None, cmap='inferno'):
    """
    Plot confusion matrix on given axes.
    
    Args:
        y_true: True labels
        y_pred: Predicted labels
        classes: List of class labels (if None, uses range from data)
        normalise: If True, normalise the confusion matrix
        ax: Matplotlib axes object (if None, creates new figure)
        cmap: Colourmap for the plot
    """
    
    # Calculate confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Normalise if requested
    if normalise:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    # Set default classes if not provided
    if classes is None:
        classes = np.unique(y_true)
    
    # Create display
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    
    # Plot on provided axes or create new figure
    disp.plot(ax=ax, cmap=cmap, colorbar=colmap, values_format='.2f' if normalise else 'd')
    
    if ax is not None:
        ax.set_xlabel('Predicted Label')
        ax.set_ylabel('True Label')
    
    return disp

### Binary example:

In [ ]:
# Actual labels: 6 passed (P), 4 failed (F)
actual = ['P', 'P', 'P', 'P', 'P', 'P', 'F', 'F', 'F', 'F']

# Predicted labels by the model
predictions = ['P', 'P', 'P', 'P', 'F', 'F', 'P', 'F', 'F', 'F']


fig, ax = plt.subplots(figsize=(8, 8))

plot_confusion_matrix(actual, predictions, 
                         classes=[0, 1], 
                         normalise=False,
                         ax=ax,
                         colmap=True,
                         cmap="Blues"
                         )
plt.show()

This can be extended to a multi-class problem. However, rather than those prediction off the diagonal being **false positives** or **negatives**, they become a false version of each class. Looking up or across each row can indicate which classes the model is mis-predicting.

Due the varying numbers of each class, it is best to normalise the score for each by dividing the each by the total of each class.

In [ ]:
fig, ax = plt.subplots(figsize=(17, 8))

plot_confusion_matrix(test_labels, all_preds, 
                         classes=range(n_classes), 
                         normalise=True,
                         ax=ax)
plt.show()

From the confusion matrix above we can see where the model has predicted well, on the diagonal. Any predictions outside of this are incorrect. 

The **9th class** (8) or *crc adenocarcinoma epithelium* has the most predictions, the majority correct, however with many that are incorrect. This can be seen by looking vertically.

In comparison, the **8th class** (7) or *cancer-associated stroma* and the **5th class** (4) or *mucus* are more likely to be predicted incorrectly, than correctly. You can see this by looking across horizontally.

### ROC-AUC Curve and Score:

The ROC curve is a plot of the true positive rate (TPR) against the false positive rate (FPR) for different threshold values. The closer the curve is to the top-left corner, the better the model is at distinguishing between the two classes. 

- **True Positive Rate (TPR, Sensitivity)**: The proportion of actual positive cases which are correctly identified.   
This is calculated as:
  $ TPR = \frac{TP}{TP + FN} $  
  where `TP` is the number of true positives and `FN` is the number of false negatives. (e.g., the percentage of sick people who are correctly identified as having the condition).

- **False Positive Rate (FPR, 1-Specificity)**: The proportion of actual negative cases which are incorrectly identified as positive.   
This is calculated as:
  $ FPR = \frac{FP}{TN + FP} $  
  where `FP` is the number of false positives and `TN` is the number of true negatives. (e.g., the percentage of healthy people who are incorrectly identified as having the condition).

These metrics help visualise how effectively a classification model distinguishes between two classes, providing insights into the model's diagnostic ability.

![FPR & TPR Terms](./images/roc_terms.png)

In a binary classification problem, the curve shows the trade-off of the positive class (1) against the negative class (0). In a multi-class classification problem, the curve shows the trade-off of one class against **all other classes**. The area-under-the-curve (AUC) score is a measure of the curve by taking the area under the curve, higher values are better.

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# get the probabilities of the model’s predictions
probs = torch.nn.functional.softmax(output, dim=1)

# plot the ROC curve for each class in the test set as a separate curve on the same plot
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111)
for i in range(n_classes):
    fpr, tpr, _ = roc_curve(test_labels.cpu().numpy() == i, probs.detach().cpu().numpy()[:, i])
    score = roc_auc_score(test_labels.cpu().numpy() == i, probs.detach().cpu().numpy()[:, i])
    ax.plot(fpr, tpr, label=f"{i}-{real_labels[i]}: {score:.2f}")

ax.plot([0, 1], [0, 1], 'k--', label='Random')

plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.show()

In the plot above the ROC curve for the *smooth muscle* (5, brown) has a rapid increase in the FPR with little increase in the TPR at 85% TPR. This indicates that the model is not good at distinguishing  approximately 15% of the smooth muscle samples from the other classes. This is highlighted by the AUC score of 0.89.

## F1 Score

An alternative to the **AUC score** is the **F1 score**, which is the harmonic mean of precision and recall, see below.

![Precision & Recall Terms By Walber - Own work, CC BY-SA 4.0, https://commons.wikimedia.org/w/index.php?curid=36926283](./images/precision-recall.png)

In a standard machine learning scenario we want to have a balance of the two types of errors, although this can depend your desired outcome. For example, when predicting cancers, you may want the recall to be high (catching as many cancers as possible) even if it means a higher false positive rate.

In [ ]:
from sklearn.metrics import f1_score

# Calculate F1 score for each class
print("\nMacro F1 Score per Class:")
print("-" * 40)
for i in range(n_classes):
    # Create binary labels for this class (one-vs-rest)
    y_true_binary = (test_labels == i).cpu().numpy().astype(int)
    y_pred_binary = (all_preds == i).cpu().numpy().astype(int)
    
    # Calculate F1 score for this class
    f1 = f1_score(y_true_binary, y_pred_binary, average='binary')
    
    print(f"Class {i} ({real_labels[i]}): {f1:.4f}")

# Calculate overall macro F1 score (average across all classes)
macro_f1 = f1_score(test_labels.cpu().numpy(), all_preds.cpu().numpy(), average='macro')
print("-" * 40)
print(f"Overall Macro F1 Score: {macro_f1:.4f}")

Class 2 and 7 both have very low F1 scores, which means the model has both high false positive and false negative rates for these classes.

In our case this can likely be solved with further training. However, if extra compute was not improving the results we could look into changing the model, oversampling the class to boost its representation in the training dataset, or change up the loss function to boost the errors for these classes.

## Summary

Using the PATH-MNIST dataset we have demonstrated how to create a CNN for a real-world problem. With more complex problems, we will need to use more complex networks, requiring more compute resources to train the larger networks.

When working in these situations it becomes more important to source GPU based compute. This will allow you to train multiple networks more quickly, comparing their results to better find the best model for your problem.

PRACTICE EXERCISE

Download the notebook and visualise the 16 feature maps produced by `model.layer1` for a single test image. The snippet below will help you extract them — your task is to plot all 16 as a 4×4 grid.

```python
# Extract the output of model.layer1 for one image
model.eval()
single_image = test_images[0].unsqueeze(0).to(device)  # add batch dimension: [1, 3, 28, 28]

with torch.no_grad():
    feature_maps = model.layer1(single_image)  # shape: [1, 16, 26, 26]

# Remove the batch dimension and move to CPU for plotting
feature_maps = feature_maps.squeeze(0).cpu()  # shape: [16, 26, 26]
print("Number of feature maps:", feature_maps.shape[0])

# Now plot the 16 feature maps as a 4×4 grid...
```

What patterns do the feature maps appear to highlight?

In [ ]:
# Write your code here:
